# Initialise GEE, and create a folium map with GEE imagery layer

In [ ]:
import ee
ee.Authenticate()

ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=ink_fnuStO1lWTjk7vtcvALHU0qtXS-Wo_FWQ5WpTqA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgxqfYaGy2zjiYqjhAf5in5VC6lS2pzOMLDuJ66SXx7YKdCuZP86ow

Successfully saved authorization token.


In [ ]:
import folium
from datetime import datetime as dt

def add_ee_layer(self, ee_image_object, vis_params, name):
  """Adds a method for displaying Earth Engine image tiles to folium map."""
  # This function is copied from the GEE tutorial
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

def image_name(ima:'ee.Image',sat:str):
  '''Define a function to name the satelitte imagery layer on the Map depending on its acquisition time'''
  date = ee.Date(ima.get('system:time_start'))
  time = date.getInfo()['value']/1000.
  name_ = sat + dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
  return name_


# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Define a map
map_1 = folium.Map(location=[16.49, 120.656], zoom_start=8, height='80%', control_scale = True)  # [lat, lon]

aoi=ee.Geometry.Point(120.656,16.49)  # [lon, lat]


In [ ]:
sen_2 = (ee.ImageCollection('COPERNICUS/S2_SR')
         .filterBounds(aoi) #[lon, lat]
         .filterDate('2021-06-01', '2021-06-30')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first())

# Add the image layer to the map and display it.
map_1.add_ee_layer(
    sen_2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, image_name(sen_2,"Sentinel 2  "))


In [ ]:
sen_2a = (ee.ImageCollection('COPERNICUS/S2_SR')
         .filterBounds(aoi) #[lon, lat]
         .filterDate('2019-06-01', '2019-06-30')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first())

# Add the image layer to the map and display it.
map_1.add_ee_layer(
    sen_2a, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, image_name(sen_2a, 'Sentinel 2  ') )


In [ ]:
lan_7 = (ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
         .filterBounds(aoi) #[lon, lat]
         .filterDate('2002-06-01', '2002-09-30')
         .sort('CLOUD_COVER')
         .first())

# Add the image layer to the map and display it.
map_1.add_ee_layer(
    lan_7, {'bands': ['SR_B3', 'SR_B2', 'SR_B1'], 'min': 7000, 'max': 13000}, image_name(lan_7,'Landsat 7  '))


In [ ]:
lan_8 = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
         .filterBounds(aoi) #[lon, lat]
         .filterDate('2014-06-01', '2014-09-30')
         .sort('CLOUD_COVER')
         .first())

# Add the image layer to the map and display it.
map_1.add_ee_layer(
    lan_8, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 7000, 'max': 13000}, image_name(lan_8, 'Landsat 8  ') )


In [ ]:
lan_8a = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
         .filterBounds(aoi) #[lon, lat]
         .filterDate('2021-06-01', '2021-09-30')
         .sort('CLOUD_COVER')
         .first())

# Add the image layer to the map and display it.
map_1.add_ee_layer(
    lan_8a, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 7000, 'max': 13000}, image_name(lan_8a,'Landsat 8  ') )  #or 6000,12000?


In [ ]:
# Adding a marker for the mining site
folium.Marker(location=[16.49, 120.656], tooltip="Santo Niño Mine").add_to(map_1) #[lat, lon]


# Add a layer control panel to the map.
map_1.add_child(folium.LayerControl())

display(map_1)


# Download the satellite image in GeoTiff through a url
cannot excess 33554432 bytes

In [ ]:
# Define an area to download
area = aoi.buffer(5000) # meters

# getDownloadURL will output the url
path = lan_8.getDownloadURL({'name':"Gee sentinel 2",'region':area})

print(path)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/a1fc06c3cfce3152e7a61716d1053642-0e78766bb440aa85b75c8861e333b35e:getPixels


# <u>Resampling </u>


In [ ]:
# Define a new map
map_2 = folium.Map(location=[16.49, 120.656], zoom_start=8, height='80%', control_scale = True)  # [lat, lon]

In [ ]:
'''ast_m = (ee.ImageCollection("ASTER/AST_L1T_003"))
ast_m.select(['B3N', 'B02', 'B01']).filterDate('2014-06-01', '2014-08-30')
# Add the image layer to the map and display it.
map_1.add_ee_layer(
    ast_m.median(), {'bands': ['B3N', 'B02', 'B01'], 'min': 1, 'max': 255}, "False Aster" )
'''
ast_ = (ee.ImageCollection("ASTER/AST_L1T_003")
  .filterBounds(aoi) #[lon, lat]
  .filterDate('2009-06-01', '2011-09-30')
  .sort('CLOUDCOVER')
  .first())

# As the Aster collection in GEE is not complete, some bands might be missing in some images
# Below is to check whether the image we selected contains the bands we trying to display or not
for a in ['B3N', 'B02', 'B01']:
  if a not in ast_.getInfo()['properties']['ORIGINAL_BANDS_PRESENT']:
    print('No required bands')
    break

map_2.add_ee_layer(
    ast_, {'bands': ['B3N', 'B02', 'B01'], 'min': 10, 'max': 255}, image_name(ast_, 'Aster  '))


# SHould be 2010-02-11 02:40:31

In [ ]:
# Nearest neighbour resampling, changing the pixel size from 15m to 10m
ast_reproject = ast_.reproject(crs='EPSG:32651',scale=10)
map_2.add_ee_layer(
    ast_reproject, {'bands': ['B3N', 'B02', 'B01'], 'min': 10, 'max': 255}, 'Nearest neighbour 15m>10m')


# The resample() method only take in arguments of 'bilinear' or 'bicubic'
# Smooth the image

# Bilinear resampling
ast_resampled_a = ast_.resample(mode='bilinear')
map_2.add_ee_layer(
    ast_resampled_a, {'bands': ['B3N', 'B02', 'B01'], 'min': 10, 'max': 255}, 'bilinear')

# Bicubic resampling (aka. cubic convoluion)
ast_resampled_b = ast_.resample('bicubic')
map_2.add_ee_layer(
    ast_resampled_b, {'bands': ['B3N', 'B02', 'B01'], 'min': 10, 'max': 255}, 'bicubic')

In [ ]:
# Adding a marker for the mining site
folium.Marker(location=[16.49, 120.656], tooltip="Santo Niño Mine").add_to(map_2) #[lat, lon]


# Add a layer control panel to the map.
map_2.add_child(folium.LayerControl())

display(map_2)

In [ ]:
# Save the map into html file. 
map_2.save('../ast resample.html')  # File location randomly made up
map_1.save('../landsat and sentinel.html')